# vision_transformer_noise_depth_kaggle.py

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import timm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import os
import json
from datetime import datetime
import time

In [2]:
print("Setting up Kaggle environment...")

Setting up Kaggle environment...


# Set device - Kaggle provides Tesla P100 or T4

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"GPU name: {torch.cuda.get_device_name() if torch.cuda.is_available() else 'CPU'}")

Using device: cuda
GPU name: NVIDIA A30


# Create output directory in Kaggle working directory

In [4]:
os.makedirs('/home/akshy_grp12/output2', exist_ok=True)

# Data transformations

In [5]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

# Load CIFAR10 dataset

In [6]:
print("Loading CIFAR10 dataset...")
train_dataset = torchvision.datasets.CIFAR10(root='/home/akshy_grp12/dataset', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='/home/akshy_grp12/dataset', train=False, download=True, transform=transform_test)

Loading CIFAR10 dataset...


# Data loaders

In [7]:
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [8]:
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

Training samples: 50000
Test samples: 10000


# Add Validation Split

In [9]:
from torch.utils.data import random_split
val_size = 5000
train_size = len(train_dataset) - val_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size])
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=2)
print(f"Validation samples: {len(val_subset)}")

Validation samples: 5000


# Chane 1

In [10]:
import numpy as np

# Sweep grids
noise_vals   = [round(x, 2) for x in np.linspace(0.5, 1.0, 11)]  # 0.00, 0.05, ..., 1.00
depth_vals   = list(range(2, 18, 2))                              # 4, 6, 8, 10, 12, 14, 16
mlp_vals     = [round(x, 2) for x in np.arange(0.5, 6.0 + 1e-9, 0.5)]  # 0.5, 1.0, ..., 6.0

EPOCHS = 30  # Or your desired value; used across training and metadata

In [11]:
def create_vit_model(depth=12, embed_dim=192, num_heads=3, mlp_ratio=4.0,
                     drop_rate=0.0, attn_drop_rate=0.0, drop_path_rate=0.0):
    model = timm.create_model(
        'vit_tiny_patch16_224',
        pretrained=False, num_classes=10,
        img_size=32, patch_size=4,
        embed_dim=embed_dim, depth=depth, num_heads=num_heads, mlp_ratio=mlp_ratio,
        qkv_bias=True, drop_rate=drop_rate, attn_drop_rate=attn_drop_rate,
        drop_path_rate=drop_path_rate
    )
    return model


In [12]:
from torchvision import transforms

CIFAR_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR_STD  = (0.2470, 0.2435, 0.2616)

_inv = transforms.Normalize(mean=[-m/s for m,s in zip(CIFAR_MEAN, CIFAR_STD)],
                            std=[1/s for s in CIFAR_STD])
_norm = transforms.Normalize(CIFAR_MEAN, CIFAR_STD)

def add_noise_in_rgb_space(x, sigma: float):
    """x normalized (B,C,H,W) or (C,H,W); sigma in [0,1] RGB scale."""
    batched = (x.dim() == 4)
    if not batched: x = x.unsqueeze(0)
    x_rgb = torch.clamp(_inv(x), 0.0, 1.0)
    x_noisy = torch.clamp(x_rgb + sigma * torch.randn_like(x_rgb), 0.0, 1.0)
    x_noisy = _norm(x_noisy)
    return x_noisy if batched else x_noisy.squeeze(0)


In [13]:
def evaluate_vit_model(model, loader, test_noise=0.0):
    """Evaluate Vision Transformer on test set."""
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    gt_all = []
    pred_all = []
    
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            
            if test_noise > 0:
                data = add_noise_in_rgb_space(data, test_noise)
            
            output = model(data)
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
            
            gt_all.extend(target.cpu().numpy())
            pred_all.extend(predicted.cpu().numpy())
    
    accuracy = 100. * correct / total
    return accuracy, gt_all, pred_all

In [14]:
def train_vit_model(model, train_loader, val_loader=None, noise_factor=0.0, epochs=30):
    """Train Vision Transformer"""
    model = model.to(device)
    model.train()
    
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.05)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    train_losses = []
    train_accuracies = []
    val_accuracies = [] if val_loader is not None else None

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for data, target in tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}', leave=False):
            data, target = data.to(device), target.to(device)
            
            if noise_factor > 0:
                data = add_noise_in_rgb_space(data, noise_factor)
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()
        
        scheduler.step()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100. * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        
        # Validation evaluation
        if val_loader is not None:
            val_acc = evaluate_vit_model(model, val_loader, test_noise=0.0)
            val_accuracies.append(val_acc)
        
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%', end='')
            if val_loader is not None:
                print(f', Val Acc: {val_accuracies[-1]:.2f}%')
            else:
                print()
    
    return train_losses, train_accuracies, val_accuracies

In [15]:
def run_experiment_v2(embed_dim=192, num_heads=3, drop_path_rate=0.1):
    """Main experiment with depth × mlp_ratio per noise, logs params & accuracy."""
    import time
    print("Starting ViT sweep… (depth, mlp_ratio) per noise")

    rows = []
    seen_param_for = {}  # (depth, mlp) -> params

    for noise in noise_vals:
        print(f"\n{'='*20} train_noise={noise} {'='*20}")
        for depth in depth_vals:
            for mlp in mlp_vals:
                # Build model
                model = create_vit_model(depth=depth, embed_dim=embed_dim,
                                         num_heads=num_heads, mlp_ratio=mlp,
                                         drop_path_rate=drop_path_rate)

                total_params = sum(p.numel() for p in model.parameters())
                seen_param_for.setdefault((depth, mlp), total_params)

                # if (PARAM_BUDGET is not None) and (total_params > PARAM_BUDGET):
                #     print(f"Skip depth={depth}, mlp={mlp} ({total_params/1e6:.1f}M > budget)")
                #     del model; torch.cuda.empty_cache() if torch.cuda.is_available() else None
                #     continue

                print(f"depth={depth:>2}, mlp={mlp:>3}, params={total_params/1e6:.2f}M, noise={noise}")

                start = time.time()
                train_losses, train_accuracies, _ = train_vit_model(
                    model, 
                    train_loader, 
                    val_loader=None, 
                    noise_factor=noise, 
                    epochs=EPOCHS
                )
                train_time = time.time() - start

                acc_clean = evaluate_vit_model(model, test_loader, test_noise=0.0)
                rows.append({
                    "train_noise": noise,
                    "depth": depth,
                    "mlp_ratio": mlp,
                    "embed_dim": embed_dim,
                    "num_heads": num_heads,
                    "params": total_params,
                    "epochs": EPOCHS,
                    "test_noise": 0.0,
                    "accuracy": acc_clean,
                    "train_time_s": round(train_time, 2)
                })

                # Save checkpoint
                torch.save(model.state_dict(),
                           f'/home/akshy_grp12/output2/vit_d{depth}_mlp{mlp}_noise{noise}.pth')

                del model
                torch.cuda.empty_cache() if torch.cuda.is_available() else None

    df = pd.DataFrame(rows)
    df.to_csv("/home/akshy_grp12/output2/vit_grid_results.csv", index=False)
    print("Saved /home/akshy_grp12/output2/vit_grid_results.csv")
    return df, seen_param_for

In [16]:
def plot_and_save_results(results, depths, noise_factors, model_params):
    """Plot and save results"""
    # Line plot
    plt.figure(figsize=(10, 6))
    colors = plt.cm.viridis(np.linspace(0, 1, len(depths)))
    
    for i, depth in enumerate(depths):
        plt.plot(noise_factors, results[depth], 
                marker='o', linewidth=2, markersize=6, 
                color=colors[i], label=f'Depth {depth}')
    
    plt.xlabel('Noise Factor')
    plt.ylabel('Test Accuracy (%)')
    plt.title('ViT: Accuracy vs Noise for Different Depths')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('/home/akshy_grp12/output2/vit_results.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Save results to CSV
    results_df = pd.DataFrame(results, index=noise_factors)
    results_df.index.name = 'Noise_Factor'
    results_df.to_csv('/home/akshy_grp12/output2/vit_results.csv')
    
    # Save parameters info
    params_df = pd.DataFrame(list(model_params.items()), columns=['Depth', 'Parameters'])
    params_df.to_csv('/home/akshy_grp12/output2/model_parameters.csv', index=False)
    
    return results_df

# 3) Evaluation that optionally adds test-time noise (keep default clean)

In [17]:
def evaluate_vit_model(model, test_loader, test_noise=0.0):
    model.eval()
    correct = total = 0
    import torch
    with torch.no_grad():
        for data, target in test_loader:
            if test_noise > 0:
                data = add_noise_in_rgb_space(data, test_noise)
            data, target = data.to(device), target.to(device)
            logits = model(data)
            pred = logits.argmax(1)
            total += target.size(0)
            correct += (pred == target).sum().item()
    return 100.0 * correct / total


In [ ]:
if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Experiment started at: {start_time}")
    
    results_df, model_params = run_experiment_v2()  # Assumes it returns df and params dict
    
    # Extract full sweep data for plotting
    depths = sorted(results_df['depth'].unique())  # e.g., [2,4,...,16]
    noise_factors = sorted(results_df['train_noise'].unique())  # Full [0.0, 0.05, ..., 1.0]
    
    # Reshape results to {depth: [accs for sorted noise_factors]}
    results = {}
    for depth in depths:
        depth_data = results_df[results_df['depth'] == depth].sort_values('train_noise')
        results[depth] = depth_data['accuracy'].values.tolist()
    
    # Plot and save (now uses full sweep)
    plot_and_save_results(results, depths, noise_factors, model_params)
    
    end_time = datetime.now()
    print(f"Experiment completed at: {end_time}")
    print(f"Total duration: {end_time - start_time}")
    
    # Print full summary
    print("\n" + "="*50)
    print("EXPERIMENT SUMMARY")
    print("="*50)
    print(results_df.groupby(['depth', 'train_noise'])['accuracy'].mean().unstack())  # Pivot table for viz
    
    # Metadata with full details
    metadata = {
        'experiment_date': start_time.isoformat(),
        'duration_seconds': (end_time - start_time).total_seconds(),
        'depths_tested': list(depths),
        'noise_factors': noise_factors,
        'epochs_per_model': 30,  # Or your EPOCHS
        'results': {str(k): dict(zip(noise_factors, v)) for k, v in results.items()},
        'model_parameters': model_params
    }
    
    with open('/home/akshy_grp12/output2/experiment_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print("\nAll results saved to /home/akshy_grp12/output2/")

Experiment started at: 2025-11-13 08:10:13.228056
Starting ViT sweep… (depth, mlp_ratio) per noise

==================== train_noise=0.5 ====================
depth= 2, mlp=0.5, params=0.40M, noise=0.5


Epoch [5/30], Loss: 1.8142, Train Acc: 33.25%


Epoch [10/30], Loss: 1.7099, Train Acc: 37.46%


Epoch [15/30], Loss: 1.6512, Train Acc: 39.59%


Epoch [20/30], Loss: 1.6047, Train Acc: 41.73%


...

Epoch [25/30], Loss: 1.5556, Train Acc: 43.48%


Epoch [30/30], Loss: 1.5375, Train Acc: 43.89%


depth= 2, mlp=1.5, params=0.54M, noise=0.5


Epoch [5/30], Loss: 1.8174, Train Acc: 33.10%


Epoch 7/30:  70%|██████████████████████████████▊             | 274/391 [00:06<00:02, 40.63it/s]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import torch
from datetime import datetime
import os

# Set style for better plots

In [ ]:
plt.style.use('seaborn-v0_8')
sns.set_palette("viridis")

In [ ]:
class ViTExperimentAnalyzer:
    def __init__(self, results_dir='/kaggle/working/outputs/'):
        self.results_dir = results_dir
        self.load_data()
        
    def load_data(self):
        """Load all experiment results"""
        try:
            # Load results CSV
            self.results_df = pd.read_csv(f'{self.results_dir}vit_results.csv', index_col='Noise_Factor')
            
            # Load model parameters
            self.params_df = pd.read_csv(f'{self.results_dir}model_parameters.csv')
            
            # Load metadata
            with open(f'{self.results_dir}experiment_metadata.json', 'r') as f:
                self.metadata = json.load(f)
                
            print("✅ Data loaded successfully!")
            print(f"Experiment date: {self.metadata['experiment_date']}")
            print(f"Duration: {self.metadata['duration_seconds']/60:.1f} minutes")
            
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            # Create sample data for demonstration
            self.create_sample_data()
    
    def create_sample_data(self):
        """Create sample data if real data isn't available"""
        print("Creating sample data for demonstration...")
        depths = [4, 6, 8]
        noise_factors = [0.0, 0.1, 0.2, 0.3]
        
        # Sample results
        self.results_df = pd.DataFrame({
            '4': [85.2, 78.5, 72.1, 65.3],
            '6': [87.5, 82.1, 76.8, 68.9],
            '8': [88.9, 84.2, 79.5, 72.1]
        }, index=noise_factors)
        self.results_df.index.name = 'Noise_Factor'
        
        # Sample parameters
        self.params_df = pd.DataFrame({
            'Depth': [4, 6, 8],
            'Parameters': [3850000, 5650000, 7450000]
        })
        
        self.metadata = {
            'experiment_date': datetime.now().isoformat(),
            'duration_seconds': 7200,
            'depths_tested': depths,
            'noise_factors': noise_factors,
            'epochs_per_model': 30
        }
    
    def basic_statistics(self):
        """Print basic statistics about the experiment"""
        print("\n" + "="*60)
        print("BASIC EXPERIMENT STATISTICS")
        print("="*60)
        
        print(f"Depths tested: {self.metadata['depths_tested']}")
        print(f"Noise factors: {self.metadata['noise_factors']}")
        print(f"Epochs per model: {self.metadata['epochs_per_model']}")
        
        print("\nModel Parameters:")
        for _, row in self.params_df.iterrows():
            print(f"  Depth {row['Depth']}: {row['Parameters']:,} parameters")
        
        print("\nPerformance Summary:")
        print(self.results_df.describe())
    
    def plot_comprehensive_analysis(self):
        """Create comprehensive visualization of results"""
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('Vision Transformer: Comprehensive Analysis of Depth vs Noise', fontsize=16, fontweight='bold')
        
        # 1. Main accuracy vs noise plot
        self._plot_accuracy_vs_noise(axes[0, 0])
        
        # 2. Performance heatmap
        self._plot_accuracy_heatmap(axes[0, 1])
        
        # 3. Performance drop analysis
        self._plot_performance_drop(axes[0, 2])
        
        # 4. Parameter efficiency
        self._plot_parameter_efficiency(axes[1, 0])
        
        # 5. Noise robustness ranking
        self._plot_robustness_ranking(axes[1, 1])
        
        # 6. Optimal depth analysis
        self._plot_optimal_depth(axes[1, 2])
        
        plt.tight_layout()
        plt.savefig(f'{self.results_dir}comprehensive_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def _plot_accuracy_vs_noise(self, ax):
        """Plot accuracy vs noise for different depths"""
        depths = self.results_df.columns.astype(int)
        noise_factors = self.results_df.index
        
        for depth in depths:
            ax.plot(noise_factors, self.results_df[str(depth)], 
                   marker='o', linewidth=2.5, markersize=8, label=f'Depth {depth}')
        
        ax.set_xlabel('Noise Factor')
        ax.set_ylabel('Test Accuracy (%)')
        ax.set_title('Accuracy vs Noise Level\nby Transformer Depth')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    def _plot_accuracy_heatmap(self, ax):
        """Plot accuracy heatmap"""
        accuracy_matrix = self.results_df.values
        im = ax.imshow(accuracy_matrix, cmap='YlOrRd', aspect='auto')
        
        plt.colorbar(im, ax=ax, label='Accuracy (%)')
        ax.set_xlabel('Transformer Depth')
        ax.set_ylabel('Noise Factor')
        ax.set_title('Accuracy Heatmap:\nDepth vs Noise')
        
        # Set ticks
        ax.set_xticks(range(len(self.results_df.columns)))
        ax.set_xticklabels([f'Depth {d}' for d in self.results_df.columns.astype(int)])
        ax.set_yticks(range(len(self.results_df.index)))
        ax.set_yticklabels(self.results_df.index)
        
        # Add accuracy values
        for i in range(len(self.results_df.index)):
            for j in range(len(self.results_df.columns)):
                ax.text(j, i, f'{accuracy_matrix[i, j]:.1f}%',
                       ha="center", va="center", color="black", fontsize=10)
    
    def _plot_performance_drop(self, ax):
        """Plot performance drop from clean to noisy data"""
        performance_drops = []
        depths = self.results_df.columns.astype(int)
        
        for depth in depths:
            clean_acc = self.results_df[str(depth)].iloc[0]  # Noise 0.0
            noisy_acc = self.results_df[str(depth)].iloc[-1]  # Highest noise
            performance_drop = clean_acc - noisy_acc
            performance_drops.append(performance_drop)
        
        bars = ax.bar(range(len(depths)), performance_drops, 
                     color=plt.cm.viridis(np.linspace(0, 1, len(depths))))
        
        ax.set_xlabel('Transformer Depth')
        ax.set_ylabel('Accuracy Drop (%)')
        ax.set_title('Performance Drop:\nClean → High Noise ({self.results_df.index[-1]})')
        ax.set_xticks(range(len(depths)))
        ax.set_xticklabels([f'Depth {d}' for d in depths])
        
        # Add value labels
        for bar, drop in zip(bars, performance_drops):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                   f'{drop:.1f}%', ha='center', va='bottom', fontsize=10)
        
        ax.grid(True, alpha=0.3, axis='y')
    
    def _plot_parameter_efficiency(self, ax):
        """Plot parameter efficiency analysis"""
        efficiency_scores = []
        depths = self.params_df['Depth'].values
        
        for depth in depths:
            avg_accuracy = np.mean(self.results_df[str(depth)])
            params_million = self.params_df[self.params_df['Depth'] == depth]['Parameters'].values[0] / 1e6
            efficiency = avg_accuracy / params_million
            efficiency_scores.append(efficiency)
        
        bars = ax.bar(range(len(depths)), efficiency_scores,
                     color=plt.cm.plasma(np.linspace(0, 1, len(depths))))
        
        ax.set_xlabel('Transformer Depth')
        ax.set_ylabel('Efficiency (Accuracy / Million Params)')
        ax.set_title('Parameter Efficiency')
        ax.set_xticks(range(len(depths)))
        ax.set_xticklabels([f'Depth {d}' for d in depths])
        
        # Add value labels
        for bar, eff in zip(bars, efficiency_scores):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                   f'{eff:.3f}', ha='center', va='bottom', fontsize=10)
        
        ax.grid(True, alpha=0.3, axis='y')
    
    def _plot_robustness_ranking(self, ax):
        """Plot noise robustness ranking"""
        robustness_scores = {}
        depths = self.results_df.columns.astype(int)
        
        for depth in depths:
            clean_acc = self.results_df[str(depth)].iloc[0]
            high_noise_acc = self.results_df[str(depth)].iloc[-1]
            robustness = (high_noise_acc / clean_acc) * 100
            robustness_scores[depth] = robustness
        
        # Sort by robustness
        sorted_depths = sorted(robustness_scores.items(), key=lambda x: x[1], reverse=True)
        depths_sorted, robustness_sorted = zip(*sorted_depths)
        
        bars = ax.bar(range(len(depths_sorted)), robustness_sorted,
                     color=plt.cm.coolwarm(np.linspace(0, 1, len(depths_sorted))))
        
        ax.set_xlabel('Transformer Depth')
        ax.set_ylabel('Robustness (% of Clean Accuracy Retained)')
        ax.set_title('Noise Robustness Ranking')
        ax.set_xticks(range(len(depths_sorted)))
        ax.set_xticklabels([f'Depth {d}' for d in depths_sorted])
        
        # Add value labels
        for bar, robustness in zip(bars, robustness_sorted):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                   f'{robustness:.1f}%', ha='center', va='bottom', fontsize=10)
        
        ax.grid(True, alpha=0.3, axis='y')
    
    def _plot_optimal_depth(self, ax):
        """Plot optimal depth analysis for different noise levels"""
        noise_levels = self.results_df.index
        optimal_depths = []
        
        for noise in noise_levels:
            accuracies = [self.results_df[str(depth)][noise] for depth in self.results_df.columns.astype(int)]
            optimal_depth = self.results_df.columns.astype(int)[np.argmax(accuracies)]
            optimal_depths.append(optimal_depth)
        
        ax.plot(noise_levels, optimal_depths, marker='s', linewidth=3, markersize=10, color='red')
        ax.set_xlabel('Noise Factor')
        ax.set_ylabel('Optimal Depth')
        ax.set_title('Optimal Transformer Depth\nby Noise Level')
        ax.set_yticks(self.results_df.columns.astype(int))
        ax.grid(True, alpha=0.3)
        
        # Add value labels
        for i, (noise, depth) in enumerate(zip(noise_levels, optimal_depths)):
            ax.annotate(f'Depth {depth}', (noise, depth), 
                       textcoords="offset points", xytext=(0,10), ha='center', fontweight='bold')
    
    def advanced_metrics_analysis(self):
        """Perform advanced metrics analysis"""
        print("\n" + "="*60)
        print("ADVANCED METRICS ANALYSIS")
        print("="*60)
        
        depths = self.results_df.columns.astype(int)
        
        # 1. Best performing models
        print("\n🏆 BEST PERFORMING MODELS:")
        best_clean = self.results_df.iloc[0].idxmax()
        best_clean_acc = self.results_df.iloc[0].max()
        print(f"  Clean data (noise=0.0): Depth {best_clean} - {best_clean_acc:.2f}%")
        
        best_noisy = self.results_df.iloc[-1].idxmax()
        best_noisy_acc = self.results_df.iloc[-1].max()
        # print(f"  Noisy data (noise=0.3): Depth {best_noisy} - {best_noisy_acc:.2f}%")
        print(f"  Noisy data (noise={self.results_df.index[-1]}): Depth {best_noisy} - {best_noisy_acc:.2f}%")
        
        best_avg = self.results_df.mean().idxmax()
        best_avg_acc = self.results_df.mean().max()
        print(f"  Average all noise levels: Depth {best_avg} - {best_avg_acc:.2f}%")
        
        # 2. Robustness analysis
        print("\n🛡️ NOISE ROBUSTNESS ANALYSIS:")
        robustness_data = []
        for depth in depths:
            clean_acc = self.results_df[str(depth)].iloc[0]
            noisy_acc = self.results_df[str(depth)].iloc[-1]
            robustness = (noisy_acc / clean_acc) * 100
            drop = clean_acc - noisy_acc
            robustness_data.append({
                'Depth': depth,
                'Clean_Acc': clean_acc,
                'Noisy_Acc': noisy_acc,
                'Robustness_%': robustness,
                'Absolute_Drop': drop
            })
        
        robustness_df = pd.DataFrame(robustness_data)
        print(robustness_df.round(2))
        
        # 3. Parameter efficiency
        print("\n⚡ PARAMETER EFFICIENCY:")
        efficiency_data = []
        for depth in depths:
            avg_acc = np.mean(self.results_df[str(depth)])
            params = self.params_df[self.params_df['Depth'] == depth]['Parameters'].values[0]
            params_million = params / 1e6
            efficiency = avg_acc / params_million
            
            efficiency_data.append({
                'Depth': depth,
                'Avg_Accuracy': avg_acc,
                'Params(M)': params_million,
                'Efficiency': efficiency
            })
        
        efficiency_df = pd.DataFrame(efficiency_data)
        print(efficiency_df.round(3))
        
        # 4. Depth scaling analysis
        print("\n📈 DEPTH SCALING ANALYSIS:")
        for i in range(1, len(depths)):
            prev_depth = depths[i-1]
            curr_depth = depths[i]
            
            clean_improvement = self.results_df[str(curr_depth)].iloc[0] - self.results_df[str(prev_depth)].iloc[0]
            avg_improvement = np.mean(self.results_df[str(curr_depth)]) - np.mean(self.results_df[str(prev_depth)])
            
            prev_params = self.params_df[self.params_df['Depth'] == prev_depth]['Parameters'].values[0]
            curr_params = self.params_df[self.params_df['Depth'] == curr_depth]['Parameters'].values[0]
            param_increase_pct = ((curr_params - prev_params) / prev_params) * 100
            
            print(f"  Depth {prev_depth}→{curr_depth}:")
            print(f"    Clean Acc: +{clean_improvement:.2f}%")
            print(f"    Avg Acc: +{avg_improvement:.2f}%")
            print(f"    Params: +{param_increase_pct:.1f}%")
    
    def generate_recommendations(self):
        """Generate practical recommendations based on analysis"""
        print("\n" + "="*60)
        print("PRACTICAL RECOMMENDATIONS")
        print("="*60)
        
        depths = self.results_df.columns.astype(int)
        
        # Find best models for different scenarios
        best_clean_depth = self.results_df.iloc[0].idxmax()
        best_noisy_depth = self.results_df.iloc[-1].idxmax()
        best_avg_depth = self.results_df.mean().idxmax()
        
        # Robustness ranking
        robustness_scores = {}
        for depth in depths:
            clean_acc = self.results_df[str(depth)].iloc[0]
            noisy_acc = self.results_df[str(depth)].iloc[-1]
            robustness_scores[depth] = (noisy_acc / clean_acc) * 100
        
        most_robust_depth = max(robustness_scores, key=robustness_scores.get)
        
        # Efficiency ranking
        efficiency_scores = {}
        for depth in depths:
            avg_acc = np.mean(self.results_df[str(depth)])
            params = self.params_df[self.params_df['Depth'] == depth]['Parameters'].values[0]
            efficiency_scores[depth] = avg_acc / (params / 1e6)
        
        most_efficient_depth = max(efficiency_scores, key=efficiency_scores.get)
        
        print("🎯 RECOMMENDED CONFIGURATIONS:")
        print(f"  • Clean Data Environment: Depth {best_clean_depth}")
        print(f"  • Noisy Data Environment: Depth {best_noisy_depth}")
        print(f"  • Balanced Performance: Depth {best_avg_depth}")
        print(f"  • Maximum Robustness: Depth {most_robust_depth}")
        print(f"  • Computational Efficiency: Depth {most_efficient_depth}")
        
        print("\n📊 KEY INSIGHTS:")
        
        # Check if deeper is better
        depth_trend = self.results_df.iloc[0].values[-1] - self.results_df.iloc[0].values[0]
        if depth_trend > 2:
            print("  ✓ Deeper models generally perform better on clean data")
        elif depth_trend < -2:
            print("  ✗ Deeper models may overfit on clean data")
        else:
            print("  ➡ Depth has minimal impact on clean data performance")
        
        # Robustness trend
        robustness_trend = robustness_scores[depths[-1]] - robustness_scores[depths[0]]
        if robustness_trend > 5:
            print("  ✓ Deeper models are more robust to noise")
        elif robustness_trend < -5:
            print("  ✗ Shallower models are more robust to noise")
        else:
            print("  ➡ Depth has minimal impact on noise robustness")
        
        # Efficiency insight
        if most_efficient_depth == depths[0]:
            print("  💡 Shallower models offer best computational efficiency")
        elif most_efficient_depth == depths[-1]:
            print("  💡 Deeper models provide good performance per parameter")
        else:
            print(f"  💡 Depth {most_efficient_depth} offers the best efficiency trade-off")
    
    def save_analysis_report(self):
        """Save comprehensive analysis report"""
        report = {
            'analysis_timestamp': datetime.now().isoformat(),
            'experiment_metadata': self.metadata,
            'summary_statistics': {
                'best_clean_accuracy': float(self.results_df.iloc[0].max()),
                'best_noisy_accuracy': float(self.results_df.iloc[-1].max()),
                'average_accuracy_by_depth': {depth: float(np.mean(self.results_df[str(depth)])) 
                                            for depth in self.results_df.columns.astype(int)},
                'performance_drops': {depth: float(self.results_df[str(depth)].iloc[0] - self.results_df[str(depth)].iloc[-1])
                                    for depth in self.results_df.columns.astype(int)}
            },
            'recommendations': self._generate_recommendations_dict()
        }
        
        with open(f'{self.results_dir}analysis_report.json', 'w') as f:
            json.dump(report, f, indent=2)
        
        print(f"\n📄 Analysis report saved to: {self.results_dir}analysis_report.json")
    
    def _generate_recommendations_dict(self):
        """Generate recommendations as dictionary"""
        depths = self.results_df.columns.astype(int)
        
        best_clean_depth = int(self.results_df.iloc[0].idxmax())
        best_noisy_depth = int(self.results_df.iloc[-1].idxmax())
        best_avg_depth = int(self.results_df.mean().idxmax())
        
        return {
            'best_for_clean_data': best_clean_depth,
            'best_for_noisy_data': best_noisy_depth,
            'best_overall_balance': best_avg_depth,
            'computational_budget_low': int(depths[0]),
            'computational_budget_high': int(depths[-1])
        }


# Run the analysis

In [ ]:
if __name__ == "__main__":
    print("🔍 Starting Vision Transformer Experiment Analysis...")
    
    # Initialize analyzer
    analyzer = ViTExperimentAnalyzer()
    
    # Run analyses
    analyzer.basic_statistics()
    analyzer.plot_comprehensive_analysis()
    analyzer.advanced_metrics_analysis()
    analyzer.generate_recommendations()
    analyzer.save_analysis_report()
    
    print("\n🎉 Analysis completed successfully!")
    print("Check the output directory for comprehensive analysis plots and reports.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

df = pd.read_csv("vit_grid_results.csv")

# Keep only evaluations on clean test for the surface (test_noise==0)
df = df[df["test_noise"] == 0.0].copy()

def plot_surface_for_noise(df_noise, noise_value):
    # Create mesh grid
    depths_sorted = sorted(df_noise["depth"].unique())
    mlps_sorted   = sorted(df_noise["mlp_ratio"].unique())
    D, M = np.meshgrid(depths_sorted, mlps_sorted, indexing='xy')  # D: depth (X), M: mlp (Y)

    # Fill Z with accuracy
    Z = np.zeros_like(D, dtype=float)
    lookup = {(int(r.depth), float(r.mlp_ratio)): float(r.accuracy) for r in df_noise.itertuples()}

    for i, d in enumerate(depths_sorted):
        for j, m in enumerate(mlps_sorted):
            Z[j, i] = lookup.get((d, m), np.nan)  # rows correspond to mlp index (j), cols to depth (i)

    # Plot
    fig = plt.figure(figsize=(7, 5))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(D, M, Z, edgecolor='none', alpha=0.85)
    ax.set_title(f"Accuracy Surface (train_noise={noise_value})")
    ax.set_xlabel("Depth")
    ax.set_ylabel("MLP Ratio")
    ax.set_zlabel("Accuracy (%)")
    plt.tight_layout()
    plt.show()

# Iterate per noise value and plot
for nv in sorted(df["train_noise"].unique()):
    plot_surface_for_noise(df[df["train_noise"] == nv], nv)
